# Initial Data Preparation

## Objectives

* Fetch data from Kaggle and save as raw data
* Initial data preparation and data cleaning
* Split data into Train, Validation, Test sets

## Inputs

* Kaggle JSON file - authentication token

## Outputs

* Generate Lemon Quality Dataset, split into Train, Validation, and Test sets



---

# Change working directory

Change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [1]:
import os
current_dir = os.getcwd()
current_dir

'/workspace/lemon-qualitycontrol/jupyter_notebooks'

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chdir() defines the new current directory

In [2]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


Confirm the new current directory

In [3]:
current_dir = os.getcwd()
current_dir

'/workspace/lemon-qualitycontrol'

## Obtain and save data from Kaggle API

Install Kaggle

In [4]:
!pip install kaggle

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.0/59.0 kB 7.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.2/78.2 kB 19.6 MB/s eta 0:00:00
  Created wheel for kaggle: filename=kaggle-1.5.12-py3-none-any.whl size=73031 sha256=d1fecf05ea8399493637899185a8210b952a20d3895eb2830412bca8e1d5971c
  Stored in directory: /home/gitpod/.cache/pip/wheels/03/f3/c7/fc5a63bb33d22177609b06c5b4c714b5eb3f1b195ce9dc5e47
Successfully built kaggle


Set Kaggle config directory environment variable to that of current working directory and set authentication to 600 to allow Kaggle package to locate JSON file

In [6]:
os.environ['KAGGLE_CONFIG_DIR'] = os.getcwd()
! chmod 600 kaggle.json

Set KaggleDataset variable as the [URL](https://www.kaggle.com/datasets/yusufemir/lemon-quality-dataset) for the dataset on Kaggle and create destination folder variable for it to be downloaded into.
Run Kaggle command to download dataset into destination folder

In [7]:
KaggleDataset = "yusufemir/lemon-quality-dataset"
DestinationFolder = "inputs/lemon-quality-dataset"
! kaggle datasets download -d {KaggleDataset} -p {DestinationFolder}


100%|████████████████████████████████████████| 233M/233M [00:03<00:00, 67.1MB/s]
100%|████████████████████████████████████████| 233M/233M [00:03<00:00, 62.8MB/s]


Unzip downloaded file and subsequently delete the originally downloaded zipped file

In [8]:
import zipfile
with zipfile.ZipFile(DestinationFolder + '/lemon-quality-dataset.zip', 'r') as zip_ref:
    zip_ref.extractall(DestinationFolder)

os.remove(DestinationFolder + '/lemon-quality-dataset.zip')

---

# Data Preparation

## Data Cleaning

Remove empty background folder and git hooks folder

In [9]:
import shutil
shutil.rmtree('inputs/lemon-quality-dataset/lemon_dataset/empty_background/')
shutil.rmtree('inputs/lemon-quality-dataset/lemon_dataset/.git/')

Check for and remove non-image files

In [10]:
def remove_non_image_file(my_data_dir):
    image_extension = ('.png', '.jpg', '.jpeg')
    folders = os.listdir(my_data_dir)
    for folder in folders:
        files = os.listdir(my_data_dir + '/' + folder)
        i = []
        j = []
        for given_file in files:
            if not given_file.lower().endswith(image_extension):
                file_location = my_data_dir + '/' + folder + '/' + given_file
                os.remove(file_location) 
                i.append(1)
            else:
                j.append(1)
                pass
        print(f"Folder: {folder} - has image file",len(j))
        print(f"Folder: {folder} - has non-image file",len(i))
    
    

In [11]:
remove_non_image_file(my_data_dir="inputs/lemon-quality-dataset/lemon_dataset")

Folder: bad_quality - has image file 951
Folder: bad_quality - has non-image file 0
Folder: good_quality - has image file 1125
Folder: good_quality - has non-image file 0


---

## Perform train-validation-test split on data

Define train-test-validation split function

In [ ]:
import random

def split_train_validation_test_images(my_data_dir, train_set_ratio, validation_set_ratio, test_set_ratio):
  
  if train_set_ratio + validation_set_ratio + test_set_ratio != 1.0:
    print("train_set_ratio + validation_set_ratio + test_set_ratio should sum 1.0")
    return

  labels = os.listdir(my_data_dir) 
  if 'test' in labels:
    pass
  else: 
    for folder in ['train','validation','test']:
      for label in labels:
        os.makedirs(name=my_data_dir+ '/' + folder + '/' + label)

    for label in labels:

      files = os.listdir(my_data_dir + '/' + label)
      random.shuffle(files)

      train_set_files_qty = int(len(files) * train_set_ratio)
      validation_set_files_qty = int(len(files) * validation_set_ratio)

      count = 1
      for file_name in files:
        if count <= train_set_files_qty:
          shutil.move(my_data_dir + '/' + label + '/' + file_name,
                      my_data_dir + '/train/' + label + '/' + file_name)
          

        elif count <= (train_set_files_qty + validation_set_files_qty ):
          shutil.move(my_data_dir + '/' + label + '/' + file_name,
                      my_data_dir + '/validation/' + label + '/' + file_name)

        else:
          shutil.move(my_data_dir + '/' + label + '/' + file_name,
                  my_data_dir + '/test/' +label + '/'+ file_name)
          
        count += 1

      os.rmdir(my_data_dir + '/' + label)
    

Apply function to lemon dataset

In [ ]:
split_train_validation_test_images(my_data_dir = f"inputs/lemon-quality-dataset/lemon_dataset",
                            train_set_ratio = 0.7,
                            validation_set_ratio = 0.1,
                            test_set_ratio = 0.2)